<span style="color: lightblue;"> 전체 구조

``` text
+---------+       +-------+      +---------------+
| START   | ---> |retriever| --> |   grade       |
+---------+       +-------+      +--------+------+
                                    |        |
                                    |문서O   | 문서X
                                    |        v
                                    v      web_search
                               generate
                                    ↓
                                   END
                                   ```

<span style="color: lightblue;"> state 는 공용 저장소
 
모든 노드가 같은 state 딕셔너리를 공유하면서 값을 바꾼다

```text
state  
 ├─ question  
 ├─ documents  
 ├─ doc_scores  
 ├─ search_type  
 └─ answer  

 ```

In [3]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import List, Literal
from typing_extensions import TypedDict
from dotenv import load_dotenv

# LangChain 관련 임포트
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# LangGraph 관련 임포트
from langgraph.graph import StateGraph, START, END

# 환경설정
load_dotenv()

if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError('key check....')


In [ ]:
def langgraph_rag():
    '''VectorDB 기반 LangGraph RAG'''
    # 상태 정의
    class RAGState(TypedDict):      # from pydantic import BaseModel 으로 불러온 BaseModel은 강제성이 있음, TypeDict은 힌트만 줄 뿐 강제성은 없음
        question:str
        documents : List[Document]
        doc_scores : List[float]    # 유사도 점수
        search_type : str
        answer : str

    # 문서 로드
    script_dir = os.getcwd()
    docs_path = os.path.join(script_dir)
    print(f'docs path : {docs_path}')

    loader = DirectoryLoader(
        docs_path,
        glob = '**/*.txt',
        loader_cls = TextLoader,
        loader_kwargs={'encoding':'utf-8'},  # 한국어면 꼭 써줄 것
    )
    docs= loader.load()

    # VectorDB 구축 -> 청킹
    text_spliter =  RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50,separators= ['\n\n','\n','.',' ',''])
    splits = text_spliter.split_documents(docs)

    vectorstores = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
        collection_name='langgraph'
    )
    print(f'VectorDB 구축 완료 청크개수 : {len(splits)}')

    # llm 초기화
    llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

    # 노드 함수들
        # 리트리버 함수
    def retrieve_node(state:RAGState)->dict:    # state:RAGState 이 의미는 state는 RAGState 형식이라는 뜻
        '''검색 노드'''
        quesiton = state['question']
        docs_with_scores = vectorstores.similarity_search_with_score(quesiton, k = 3)
        documents =  [ doc for doc,score in docs_with_scores]
        scores =  [ 1-score for doc,score in docs_with_scores]

        print(f' [retriever] {len(documents)}개 문서 검색됨')        
        return {'documents': documents, 'doc_scores':scores, 'search_type':'internal'}   # state 업데이트

    def grade_documents_node(state:RAGState)->dict:
        '''문서평가 노드'''
        threshold = 0.3 # -> 유사도 점수 최소 기준값
        filtered_docs, filtered_scores = [],[]  # 유사도 최소 기준값 조건을 통과한 문서만 담을 빈 리스트
        for doc, score in zip(state['documents'],state['doc_scores']): # state['documents'] (리트리버가 반환한 문서들)/ state['doc_scores'] (그 문서들의 유사도 점수)
            if score >= threshold:  # score가 0.3 이상인 것들만 통과
                filtered_docs.append(doc); filtered_scores.append(score) # 통과한 문서들을 더해준다     # ; 두 줄을 한줄로 합친것
        print(f"[grade] {len(state['documents'])}개 --> {len(filtered_docs)}개 문서 유지")
        return {'documents' : filtered_docs, 'doc_scores':filtered_scores}

    def web_search_node(state:RAGState)->dict:
        '''웹검색 노드(시뮬레이션)'''
        web_result = Document(
            page_content=f"웹 검색 결과 : {state['question']}에 대한 최신 결과 입니다.",
            metadata = {'source':'web_search'}
        )
        return {'document':[web_result],'doc_scores':[0.8], 'search_type':'web'}

    def generate_node(state:RAGState)->dict:
        '''생성노드'''  
        context = '\n'.join([ doc.page_content for doc in state['documents']])
        prompt = ChatPromptTemplate.from_messages([
            ('system','제공된 문맥을 바탕으로 한국어로 답변하세요'),
            ('human', '문맥:\n{context}\n\n질문:{question}\n\n답변:')
        ])
        chain = prompt | llm | StrOutputParser()
        answer = chain.invoke({'context':context, 'question' : state['question'] })
        return {'answer':answer}
    
    # 조건 함수
    def decide_to_generate(state:RAGState)-> Literal['generate','web_search']:
        '''조건부 분기 함수'''    
        if state['documents'] and len(state['documents']) > 0:
            return 'generate'
        else:
            return 'web_search'

    # 그래프 구축(add_node  add_edge  add_conditional_edges)
    graph = StateGraph(RAGState)
    graph.add_node('retriever',retrieve_node)
    graph.add_node('grade',grade_documents_node)
    graph.add_node('web_search',web_search_node)
    graph.add_node('generate',generate_node)

    graph.add_edge(START, 'retriever')
    graph.add_edge('retriever', 'grade')
    graph.add_conditional_edges(
        'grade',
        decide_to_generate,
        { 'generate':'generate', 'web_search': 'web_search'}
    )
    graph.add_edge('web_search', 'generate')    # 웹 검색을 바탕으로 최종 답변을 생성
    graph.add_edge('generate', END)
    # 그래프 컴파일

    app = graph.compile()       # 노드와 엣지를 내부적으로 연결. 노드와 엣지 설정 후 꼭 써줘야 실행함
    # 그래프 invoke(질문)
    test_qeustion = [
        'LangGraph의 핵심 개념을 설명해 주세요',
        'RAG란 무엇인가요?',
        '오늘 서울 날씨는 어떤가요?'  # 내부 문서에 없음
    ]
    # 각 질문에 대한 출력
    for question in test_qeustion:        
        result = app.invoke({
            'question':question,
            'documents' : [],
            'doc_scores' : [],
            'search_type' : "",
            'answer' : ""
        })

        print(f'\n 답변 :\n {result['answer']}')
        print(f'\n 검색유형 :{result['search_type']}, 참조문서 : {len(result['documents'])}개')


if __name__ == '__main__':
    langgraph_rag()


docs path : c:\python_src\6.openAI
VectorDB 구축 완료 청크개수 : 21
 [retriever] 3개 문서 검색됨
[grade] 3개 --> 1개 문서 유지

 답변 :
 LangGraph의 핵심 개념은 다음과 같습니다:

1. **State(상태)**: 에이전트의 현재 상태를 나타내는 데이터 구조로, 에이전트가 수행하는 작업이나 프로세스의 진행 상황을 기록합니다. 이 상태는 에이전트가 다음에 어떤 행동을 취할지를 결정하는 데 중요한 역할을 합니다.

2. **Node(노드)**: 실제 작업을 수행하는 함수로, 특정 작업이나 기능을 실행하는 단위입니다. 각 노드는 특정한 기능을 가지고 있으며, 에이전트가 수행해야 할 작업을 정의합니다.

3. **Edge(엣지)**: 노드 간의 제어 흐름을 정의하는 요소로, 한 노드에서 다른 노드로의 전환을 나타냅니다. 엣지는 작업의 순서를 결정하고, 노드 간의 관계를 형성합니다.

4. **Conditional Edge**: 조건에 따라 다른 노드로 분기하는 엣지로, 특정 조건이 충족되면 다른 경로로 진행할 수 있게 합니다. 이를 통해 에이전트는 상황에 맞는 적절한 행동을 선택할 수 있습니다.

이러한 개념들은 LangGraph의 구조와 동작 방식을 이해하는 데 필수적이며, 에이전트가 복잡한 작업을 효율적으로 수행할 수 있도록 돕습니다.

 검색유형 :internal, 참조문서 : 1개
 [retriever] 3개 문서 검색됨
[grade] 3개 --> 0개 문서 유지

 답변 :
 RAG는 "Retrieval-Augmented Generation"의 약자로, 정보 검색과 생성 모델을 결합한 접근 방식을 의미합니다. 이 방법은 주어진 질문이나 요청에 대해 관련 정보를 검색한 후, 그 정보를 바탕으로 자연어로 응답을 생성하는 방식입니다. RAG는 특히 대규모 데이터베이스나 문서에서 필요한 정보를 효과적으로 찾아내고, 이를 활용하여 더 정확하고 풍부한 답변을 제공하는 데 유용합니다. 이 기술은 